In [3]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import IPython.display as display
from tqdm import tqdm
import random

In [6]:
d = {}
d['dd'] = 0
type(d)

dict

In [22]:
# 여러 개의 폴더에 있는 것 합치기
images, label_list = [], []
path = '../data/images/'
for folder in os.listdir(path):
    for image in tqdm(os.listdir(path+folder), desc=folder+" 폴더 작업중"):
        if '.370' in folder:
            images.append({'path': path+folder+"/"+image, 'class': 0})
            label_list.append(0)
        elif '.817' in folder:
            images.append({'path': path+folder+"/"+image, 'class': 1})
            label_list.append(1)
# print('num_classes: ', len(set(label_list)))
# encoder = class_one_hot_encoder(label_list)

20220713221533.817 폴더 작업중: 100%|██████████| 39/39 [00:00<00:00, 38956.38it/s]


In [33]:
image_class = images[1]['class']
print(image_class)
image_file = tf.io.decode_image(tf.io.read_file(images[1]['path'])).numpy()
image_file = tf.image.resize(image_file, size=[256, 256])
image_file = tf.io.encode_jpeg(tf.cast(image_file, dtype=tf.uint8)).numpy()
record = tf.train.Example(
    features=tf.train.Features(
        feature={
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_file])),
            'class': tf.train.Feature(int64_list=tf.train.Int64List(value=[image_class])),
            # 'bb_box': tf.train.Feature(float_list=tf.train.FloatList(value=bb_box_list)),
            # 'landmark': tf.train.Feature(float_list=tf.train.FloatList(value=landmark_list))
        }
    )
)
# f.write(record.SerializeToString())

1


In [34]:
# write tfrecord
save_path = '../data/face_data.tfrecord'
i = 0
random.shuffle(images)
with tf.io.TFRecordWriter(save_path) as f:
    for image in tqdm(images):

        image_class = image['class']
        # image_class = image_class.astype('int64')

        try:
            # image
            image_file = tf.io.decode_image(tf.io.read_file(image['path'])).numpy()
            image_file = tf.image.resize(image_file, size=[256, 256])
            image_file = tf.io.encode_jpeg(tf.cast(image_file, dtype=tf.uint8)).numpy()
            # image_width = int(image['@width'])
            # image_height = int(image['@height'])

            # image_box = image['box']
            # top = int(image_box['@top'])
            # left = int(image_box['@left'])
            # width = int(image_box['@width'])
            # height = int(image_box['@height'])
            # bb_box_list = [top, left, width, height]
            # bb_box_list = point_adjust(bb_box_list, image_width, image_height)

            # # 'head_top', 'lear_base', 'lear_tip', 'leye, 'nose', 'rear_base', 'rear_tip', 'reye'
            # # x,y 순서
            # landmark_list = []
            # for point in image_box['part']:
            #     x = int(point['@x'])
            #     y = int(point['@y'])
            #     landmark_list.extend([x, y])
            # landmark_list = point_adjust(landmark_list, image_width, image_height)

            # record
            record = tf.train.Example(
                features=tf.train.Features(
                    feature={
                        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_file])),
                        'class': tf.train.Feature(int64_list=tf.train.Int64List(value=[image_class])),
                        # 'bb_box': tf.train.Feature(float_list=tf.train.FloatList(value=bb_box_list)),
                        # 'landmark': tf.train.Feature(float_list=tf.train.FloatList(value=landmark_list))
                    }
                )
            )
            f.write(record.SerializeToString())
            i += 1
        except Exception as e:
            print(e, i)
            continue

print('이미지 갯수: ', i)


100%|██████████| 203/203 [00:07<00:00, 27.69it/s]

이미지 갯수:  203


In [35]:
filenames = '../data/face_data.tfrecord'
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [40]:
# Create a dictionary describing the features.
image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),    
    'class': tf.io.FixedLenFeature([], tf.int64),    
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_dataset.map(_parse_image_function)
parsed_image_dataset

<MapDataset element_spec={'class': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
for image_features in parsed_image_dataset:
    image_raw = image_features['image'].numpy()
display.display(display.Image(data=image_raw))